In [2]:
import os
import sys
from datetime import datetime
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 --executor-memory 3g --driver-memory 2g pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as t
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
import numpy as np
from numpy import dot
from numpy. linalg import norm
conf = SparkConf()
import json
import re
import pandas as pd

In [3]:
spark = SparkSession\
    .builder\
    .config(conf=conf)\
    .appName("GF_spark")\
    .getOrCreate()

In [4]:
spark

In [5]:
sc = spark.sparkContext

In [6]:
sc

<SparkContext master=yarn appName=GF_spark>

In [6]:
! hdfs dfs -ls /labs/slaba04/

Found 1 items
-rw-r--r--   3 hdfs hdfs  655090069 2022-01-06 18:46 /labs/slaba04/gender_age_dataset.txt


In [12]:
df.groupBy('gender').count().rdd.getNumPartitions()

200

In [28]:
df = spark.read.csv('/labs/slaba04/gender_age_dataset.txt', sep='\t', header=True)

In [36]:
df.select(F.lit(1).cast(t.ByteType()), 'gender').printSchema()

root
 |-- CAST(1 AS TINYINT): byte (nullable = false)
 |-- gender: string (nullable = true)



In [8]:
df.show(3, vertical=True)

-RECORD 0-------------------------
 gender    | F                    
 age       | 18-24                
 uid       | d50192e5-c44e-4ae... 
 user_json | {"visits": [{"url... 
-RECORD 1-------------------------
 gender    | M                    
 age       | 25-34                
 uid       | d502331d-621e-472... 
 user_json | {"visits": [{"url... 
-RECORD 2-------------------------
 gender    | F                    
 age       | 25-34                
 uid       | d50237ea-747e-48a... 
 user_json | {"visits": [{"url... 
only showing top 3 rows



In [9]:
df.printSchema()

root
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)
 |-- uid: string (nullable = true)
 |-- user_json: string (nullable = true)



In [10]:
df.select('gender').distinct().show()
df.select('age').distinct().show()

+------+
|gender|
+------+
|     F|
|     M|
|     -|
+------+

+-----+
|  age|
+-----+
| >=55|
|45-54|
|    -|
|35-44|
|25-34|
|18-24|
+-----+



In [11]:
df = df.filter((F.col('age') != '-') & (F.col('gender') != '-'))

In [12]:
df.select('gender').distinct().show()
df.select('age').distinct().show()

+------+
|gender|
+------+
|     F|
|     M|
+------+

+-----+
|  age|
+-----+
| >=55|
|45-54|
|35-44|
|25-34|
|18-24|
+-----+



In [13]:
def www(x):
    l = []
    x = json.loads(x)['visits']
    for i in x:
        a = i['url'].strip()
        b = re.search("(?:http[s]?:\/\/)+(?:www\.)?([^\/]*)",a)
        if b:
            l.append(b.group(1))
    return l

In [14]:
list_n = df.rdd.map(lambda x: (x[0], x[1])).distinct().map(lambda x: ([x[0], x[1]])).collect()

In [15]:
di_1 = {0: "M", 1: 'F'}
di_2 = {0: '18-24', 1: '25-34',2: '35-44' ,3: '45-54', 4: '>=55'}

In [16]:
def g_a(x):
    for i in di_2:
        if x == di_2[i]:
            return i

In [17]:
def g_s(x):
    for i in di_1:
        if x == di_1[i]:
            return i

In [18]:
df.show()

+------+-----+--------------------+--------------------+
|gender|  age|                 uid|           user_json|
+------+-----+--------------------+--------------------+
|     F|18-24|d50192e5-c44e-4ae...|{"visits": [{"url...|
|     M|25-34|d502331d-621e-472...|{"visits": [{"url...|
|     F|25-34|d50237ea-747e-48a...|{"visits": [{"url...|
|     F|25-34|d502f29f-d57a-46b...|{"visits": [{"url...|
|     M| >=55|d503c3b2-a0c2-4f4...|{"visits": [{"url...|
|     F|25-34|d5090ddf-5648-487...|{"visits": [{"url...|
|     F|25-34|d50bcef8-16ff-4e8...|{"visits": [{"url...|
|     F|18-24|d50e23dc-0cbd-488...|{"visits": [{"url...|
|     F|45-54|d50fdabb-4208-441...|{"visits": [{"url...|
|     F|18-24|d511b480-23a6-482...|{"visits": [{"url...|
|     F|25-34|d51294ed-1b95-4e4...|{"visits": [{"url...|
|     F|25-34|d512e295-6a85-491...|{"visits": [{"url...|
|     M|25-34|d51441ea-9dda-454...|{"visits": [{"url...|
|     F|25-34|d51822d4-105b-457...|{"visits": [{"url...|
|     F|35-44|d5183db2-c8e5-413

In [19]:
rdd = df.rdd.map(lambda x: (g_s(x[0]), g_a(x[1]), x[2], www(x[3])))

schema = t.StructType(fields=[t.StructField("gender", t.IntegerType()),
                            t.StructField("age", t.IntegerType()),
                            t.StructField("uid", t.StringType()),
                             t.StructField("fff", t.ArrayType(t.StringType()))])

In [20]:
rdd.take(1)

[(1,
  0,
  'd50192e5-c44e-4ae8-ae7a-7cfe67c8b777',
  ['zebra-zoya.ru',
   'news.yandex.ru',
   'sotovik.ru',
   'news.yandex.ru',
   'sotovik.ru'])]

In [21]:
df_pre = spark.createDataFrame(rdd, schema=schema)
df_pre.printSchema()
df_pre.show(5)

root
 |-- gender: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- uid: string (nullable = true)
 |-- fff: array (nullable = true)
 |    |-- element: string (containsNull = true)

+------+---+--------------------+--------------------+
|gender|age|                 uid|                 fff|
+------+---+--------------------+--------------------+
|     1|  0|d50192e5-c44e-4ae...|[zebra-zoya.ru, n...|
|     0|  1|d502331d-621e-472...|[sweetrading.ru, ...|
|     1|  1|d50237ea-747e-48a...|[ru.oriflame.com,...|
|     1|  1|d502f29f-d57a-46b...|[translate-tattoo...|
|     0|  4|d503c3b2-a0c2-4f4...|[mail.rambler.ru,...|
+------+---+--------------------+--------------------+
only showing top 5 rows



In [22]:
from pyspark.ml.feature import HashingTF
hashingTF = HashingTF(inputCol = 'fff', outputCol = 'features')
output = hashingTF.transform(df_pre)
#idf = IDF(inputCol='features_2', outputCol='features')
#idfModel = idf.fit(output)
#output = idfModel.transform(output)

In [23]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import RandomForestClassifier
#lr_gender = LogisticRegression(featuresCol='features',labelCol='gender',maxIter=15, regParam=2.2)
#lr_age = LogisticRegression(featuresCol='features',labelCol='age',maxIter=15, regParam=2.2)
lr_gender = RandomForestClassifier(featuresCol='features',labelCol='gender')
lr_age = RandomForestClassifier(featuresCol='features',labelCol='age')

In [24]:
#from pyspark.ml.classification import RandomForestClassifier
#lr_gender = RandomForestClassifier(numTrees=5, maxDepth=8, labelCol="gender", seed=42, featuresCol="features")
#lr_age = RandomForestClassifier(numTrees=5, maxDepth=8, labelCol="age", seed=42, featuresCol="features")

In [25]:
#%time train, test = output.randomSplit([0.9, 0.1])
%time model_gender = lr_gender.fit(output)
%time model_age = lr_age.fit(output)

CPU times: user 45.1 ms, sys: 45.8 ms, total: 90.9 ms
Wall time: 4min 27s
CPU times: user 61.9 ms, sys: 19.2 ms, total: 81.1 ms
Wall time: 4min 20s


In [27]:
type(model_gender)

pyspark.ml.classification.RandomForestClassificationModel

# stop

In [88]:
#predictions = model_gender.transform(test).select('gender', 'age', 'features',F.col('prediction').alias('prediction_gender'))
#predictions = model_age.transform(predictions)\
#    .select('gender', 'age', F.col('prediction_gender'), F.col('prediction').alias('prediction_age'))
#predictions.printSchema()

In [89]:
#predictions.filter((F.col('gender') == F.col('prediction_gender'))).count()

In [90]:
#predictions.filter((F.col('age') == F.col('prediction_age'))).count()

In [62]:
#aa = predictions.filter((F.col('age') == F.col('prediction_age')) & (F.col('gender') == F.col('prediction_gender'))).count()

In [63]:
#bb = predictions.count()

In [30]:
#aa / bb

0.2631578947368421

In [113]:
! pwd

/data/home/sergey.vaschuk


## kafka

In [1]:
KAFKA_BOOTSTRAP_SERVER = 'spark-node-1.newprolab.com:6667'
KAFKA_INPUT_TOPIC = 'input_sergey.vaschuk'
KAFKA_OUTPUT_TOPIC = 'sergey.vaschuk'

def create_console_sink(df):
    return df \
            .writeStream \
            .format("console") \
            .trigger(processingTime="10 seconds") \
            .option("truncate", "false")
read_kafka_params = {
    "kafka.bootstrap.servers": 'spark-master-1.newprolab.com:6667',
    "subscribe": "input_sergey.vaschuk",
    "startingOffsets": "latest"
}

def clean_df(df):
    event_schema = t.StructType([
        t.StructField('uid', t.StringType(), True),
        t.StructField('visits', t.StringType(), True),
    ])


    visit_schema = t.ArrayType(
        t.StructType([
            t.StructField('url', t.StringType(), True),
            #t.StructField('timestamp', t.LongType(), True)
        ])
    )


    df = (
        df
        .select(F.col('value').cast('string').alias('value'))
        .select(F.from_json(F.col('value'), event_schema).alias('event'))
        .select(
            'event.uid', 
            F.from_json(F.col('event.visits'), visit_schema).alias('visits')
        )
    )
    return df
def kill_all():
    streams = SparkSession.builder.getOrCreate().streams.active
    if streams:
        for s in streams:
            desc = s.lastProgress["sources"][0]["description"]
            s.stop()
            print("Stopped {s}".format(s=desc))

In [117]:
kill_all()

Stopped KafkaV2[Subscribe[input_sergey.vaschuk]]


# GO GO GO

In [7]:
kafka_stream = (
    spark
    .readStream
    .format('kafka')
    .option('kafka.bootstrap.servers', KAFKA_BOOTSTRAP_SERVER)
    .option('subscribe', KAFKA_INPUT_TOPIC)
    #.option('startingOffsets', 'earliest')
    .option('failOnDataLoss', 'False')
    .load()
)


In [9]:
type(kafka_stream)

pyspark.sql.dataframe.DataFrame

In [116]:
kafka_stream = (
    spark
    .readStream
    .format('kafka')
    .option('kafka.bootstrap.servers', KAFKA_BOOTSTRAP_SERVER)
    .option('subscribe', KAFKA_INPUT_TOPIC)
    #.option('startingOffsets', 'earliest')
    .option('failOnDataLoss', 'False')
    .load()
)


df = clean_df(kafka_stream).withColumn('url', F.col('visits').getItem('url')).select('uid', F.explode('url').alias('url'))\
    .withColumn('url_host', F.expr("parse_url(url, 'HOST')"))\
    .groupBy('uid').agg(F.collect_list('url_host').alias('visits'))

hashingTF = HashingTF(inputCol = 'visits', outputCol = 'features')
output = hashingTF.transform(df)

predictions = model_age.transform(output).select('uid', 'features', F.col('prediction').alias('prediction_age'))
predictions = model_gender.transform(predictions).select('uid', F.col('prediction').alias('prediction_gender'), 'prediction_age')
predictions_df = predictions.withColumn('gender', F.when(F.col('prediction_gender').isin(0), 'M').otherwise('F'))\
    .withColumn('age', F.when(F.col('prediction_age').isin(0), '18-24')\
               .when(F.col('prediction_age').isin(1), '25-34')\
               .when(F.col('prediction_age').isin(2), '35-44')\
               .when(F.col('prediction_age').isin(3), '45-54')\
               .when(F.col('prediction_age').isin(4), '>=55'))\
    .select('uid', 'gender', 'age')

kafka_out_df = (
    predictions_df
    .select(F.to_json(F.struct(*predictions_df.columns)).alias('value'))
)

kafka_write_stream = (
    kafka_out_df
    .writeStream
    .format('kafka')
    .option('kafka.bootstrap.servers', KAFKA_BOOTSTRAP_SERVER)
    .outputMode("complete")
    .option("checkpointLocation", "checkpoints/checkpoints_lab04")
    .option('topic', KAFKA_OUTPUT_TOPIC)
    .start()
)

# ============================================

# батч

In [29]:
kafka_read_df = (
    spark.read
    .format('kafka')
    .option('kafka.bootstrap.servers', KAFKA_BOOTSTRAP_SERVER)
    .option('subscribe', KAFKA_INPUT_TOPIC)
  #  .option('startingOffsets', "latest")
    .option('failOnDataLoss', 'False')
    .load()
    #.cache()
)

#test_df = spark.read.parquet('/user/sergey.vaschuk/lab_TEST/')
df = clean_df(kafka_read_df).withColumn('url', F.col('visits').getItem('url')).select('uid', F.explode('url').alias('url'))\
    .withColumn('url_host', F.expr("parse_url(url, 'HOST')"))\
    .groupBy('uid').agg(F.collect_list('url_host').alias('visits'))

hashingTF = HashingTF(inputCol = 'visits', outputCol = 'features')
output = hashingTF.transform(df)
#idf = IDF(inputCol='features_2', outputCol='features')
#idfModel = idf.fit(output)
#output = idfModel.transform(output)

predictions = model_age.transform(output).select('uid', 'features', F.col('prediction').alias('prediction_age'))
predictions = model_gender.transform(predictions).select('uid', F.col('prediction').alias('prediction_gender'), 'prediction_age')
predictions_df = predictions.withColumn('gender', F.when(F.col('prediction_gender').isin(0), 'M').otherwise('F'))\
    .withColumn('age', F.when(F.col('prediction_age').isin(0), '18-24')\
               .when(F.col('prediction_age').isin(1), '25-34')\
               .when(F.col('prediction_age').isin(2), '35-44')\
               .when(F.col('prediction_age').isin(3), '45-54')\
               .when(F.col('prediction_age').isin(4), '>=55'))\
    .select('uid', 'gender', 'age')

kafka_out_df = (
    predictions_df
    .select(F.to_json(F.struct(*predictions_df.columns)).alias('value'))
)
(
    kafka_out_df
    .write
    .format('kafka')
    .option('kafka.bootstrap.servers', KAFKA_BOOTSTRAP_SERVER)
    .option('topic', KAFKA_OUTPUT_TOPIC)
    .save()
)

In [38]:
%time predictions_df.show()

+--------------------+------+-----+
|                 uid|gender|  age|
+--------------------+------+-----+
|0108d217-e476-493...|     F|35-44|
|0192cc54-559c-4c8...|     M|45-54|
|019acd5e-be9a-4cd...|     F|18-24|
|02e7f830-da57-4d5...|     F|35-44|
|1d160259-73d8-451...|     F|25-34|
|1e14a504-276e-448...|     F|18-24|
|1eb313db-34ff-4bf...|     M|25-34|
|1eff6e4f-3b8a-447...|     M|25-34|
|3e75c432-cb78-488...|     M|35-44|
|47565df3-13e3-460...|     M|25-34|
|4766a8ab-e9b6-4e0...|     F|35-44|
|50637c81-fffa-4ee...|     F|25-34|
|5a023519-f28e-4eb...|     M|25-34|
|5a781caa-6131-4d9...|     F|25-34|
|5ab3c7b8-c550-493...|     F|35-44|
|7302e78a-ec04-47e...|     F|18-24|
|73081df3-8f41-435...|     F|25-34|
|89fe85cb-ea4c-4be...|     M|25-34|
|8affa6ce-24c7-4ed...|     M| >=55|
|b2e4450d-c582-441...|     F|35-44|
+--------------------+------+-----+
only showing top 20 rows

CPU times: user 11.4 ms, sys: 912 µs, total: 12.3 ms
Wall time: 2min 30s


In [118]:
spark.stop()